Cartar de aclaracion de posible desvío

In [1]:
import pandas as pd
import collections

from docxtpl import DocxTemplate

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
ruta

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
#path_xlsx_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V3.xlsx'
path_xlsx_avance = r'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V3.xlsx'

In [5]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

C:\Users\bismarksr\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\bismarksr\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [6]:
xlsx_entregas.head(3)

,INST,GRUPO,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,PAQ,TM. ENTREGADAS,AVANCE,INICIO DE ENTREGAS,ULTIMA ENTREGA,COD_COSECHA
0,1,16,644,ANTELO ANTELO ARIEL,555.36,104,568.61,1.023858,NaN,NaN,262
1,1,6,660,AGUILERA ANTELO ALEX ENRIQUE,800.00,104,689.52,0.861900,NaN,NaN,49
2,1,5,691,AGUILAR POLARES JOSE,752.70,104,1237.58,1.644188,NaN,NaN,57


In [7]:
xlsx_avance.head(3)

,COD_COS,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO,ENTREGA,%_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,...,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROM_REST,DIFERENCIA
0,1,515.0,AGUILERA OLGA RIVERO VDA DE,26449.77,26112.72,0.987257,543.39,420.85,122.54,0.77449,...,0.212767,543.693782,55.390276,30115.348382,24326.385166,1786.334834,0.931591,5788.963216,337.05,5451.913216
1,Grand Total,NaN,NaN,26449.77,26112.72,0.987257,543.39,420.85,122.54,0.77449,...,0.212767,543.693782,55.390276,30115.348382,24326.385166,1786.334834,0.931591,5788.963216,337.05,5451.913216


## Argumentos

In [9]:
fecha_inf = '07 de octubre de 2023'
fecha_form = '07-10-2023' # fecha para el nombre de archivo

fecha_aclaracion = '10 de septiembre de 2023'
cod_grupos = [1]

In [ ]:
for i in cod_grupos:
    # datos del grupo
    grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == i].copy()
    grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
    grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
    grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
    
    # convertir datos del grupo a objetos
    list_obj_caneros = []
    for index, row in grupo.iterrows():
        canero = {
            'inst' : row['INST'],
            'cod_ca' : row['CODIGO CAÑERO '],
            'nom_ca' : row['NOMBRE CAÑERO'],
            'compromiso' : row['COMPROMISO DE ENTREGA'],
            'entrega' : row['TM. ENTREGADAS'],
            'entrega_porcen' : row['avance%']
        }
        list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
    
    # totales de los datos del grupo
    grupo_totales = {
        'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
        'entregas' : grupo['TM. ENTREGADAS'].sum(),
        'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
    }
    obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
    
    # datos generales de avance de cosecha
    avance = xlsx_avance[xlsx_avance['COD_COS'] == i]
    avance_totales = {
        'fecha_aclaracion' : fecha_aclaracion,
        'fecha_informe' : fecha_inf,
    
        'entrega_porcen' : float(avance['%_ENTREGA']) * 100,
        'ha_canha' : float(avance['TOTAL_AREA']),
        'ha_cosecha' : float(avance['COSECHA']),
        'cosecha_porcen' : float(avance['%COSECHA']) * 100,
        'diferencia_porcen' : float(avance['DIF.AVANCE']) * 100,
        
        'compromiso_rest' : float(avance['COMPROM_REST']),
        'tn_en_campo' : float(avance['TN_EN_CAMPO']),
        'diferencia' : float(avance['DIFERENCIA'])
    }
    
    # datos generales a objeto
    obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
    
    # generar reporte
    doc = DocxTemplate(ruta + '/templates/tpl_informe_aclarativa_desvio.docx')
    context = {
        'insp' : list_obj_caneros,
        'grupo_totales' : obj_grupo_totales,
        'avance' : obj_avance_totales
    }
    doc.render(context)
    doc.save(ruta + '/informes/_' + str(i) + '_RDC_' + fecha_form + '.docx')
    print('Se generó reporte de grupo:', i)

Se generó reporte de grupo: 1


## DATOS DEL GRUPO

In [18]:
cod_grupo = 1

In [19]:
grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == cod_grupo].copy()
grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
grupo

,INST,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,TM. ENTREGADAS,avance%
393,50,515,AGUILERA OLGA RIVERO VDA DE,12273.83,9036.62,73.625103
401,50,1215,AGUILERA RIVERO OLGA KARINA,3034.81,4017.21,132.371055
440,50,8003,AGUILERA RIVERO SERGIO STEFAN,3553.99,4015.77,112.993284
472,50,40004,AGUILERA RIVERO NICOLAS JULIO,3500.04,4076.59,116.472669
584,62,42163,GARCIA PELLEGRINI LUIS FERNANDO,514.39,955.65,185.783161
801,69,790,AGUILERA RIVERO JESSICA BEATRIZ,3572.71,4010.88,112.264360


In [20]:
list_obj_caneros = []
for index, row in grupo.iterrows():
    canero = {
        'inst' : row['INST'],
        'cod_ca' : row['CODIGO CAÑERO '],
        'nom_ca' : row['NOMBRE CAÑERO'],
        'compromiso' : row['COMPROMISO DE ENTREGA'],
        'entrega' : row['TM. ENTREGADAS'],
        'entrega_porcen' : row['avance%']
    }
    list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_caneros

[canero(inst=50, cod_ca=515, nom_ca='AGUILERA OLGA RIVERO VDA DE ', compromiso=12273.83, entrega=9036.62, entrega_porcen=73.62510316665622),
 canero(inst=50, cod_ca=1215, nom_ca='AGUILERA RIVERO OLGA KARINA ', compromiso=3034.81, entrega=4017.21, entrega_porcen=132.37105453059667),
 canero(inst=50, cod_ca=8003, nom_ca='AGUILERA RIVERO SERGIO STEFAN ', compromiso=3553.99, entrega=4015.77, entrega_porcen=112.99328360518741),
 canero(inst=50, cod_ca=40004, nom_ca='AGUILERA RIVERO NICOLAS JULIO ', compromiso=3500.04, entrega=4076.59, entrega_porcen=116.47266888378418),
 canero(inst=62, cod_ca=42163, nom_ca='GARCIA PELLEGRINI LUIS FERNANDO', compromiso=514.39, entrega=955.65, entrega_porcen=185.78316063687086),
 canero(inst=69, cod_ca=790, nom_ca='AGUILERA RIVERO JESSICA BEATRIZ ', compromiso=3572.71, entrega=4010.88, entrega_porcen=112.26435954779426)]

In [21]:
grupo_totales = {
    'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
    'entregas' : grupo['TM. ENTREGADAS'].sum(),
    'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
}
obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

grupo_totales(compromiso=26449.769999999997, entregas=26112.720000000005, entrega_porcen=98.72569780379946)

## DATOS DE AVANCE

In [22]:
xlsx_avance.head(3)

,COD_COS,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO,ENTREGA,%_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,...,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROM_REST,DIFERENCIA
0,1,515,AGUILERA OLGA RIVERO VDA DE,26449.77,26112.72,0.987257,543.39,420.85,122.54,0.774490,...,0.212767,543.693782,55.390276,30115.348382,24326.385166,1786.334834,0.931591,5788.963216,337.05,5451.913216
1,2,1180,BALCAZAR BALCAZAR JESUS E.,10848.49,11676.37,1.076313,170.47,159.76,10.71,0.937174,...,0.139139,170.249063,50.971072,8677.777218,8200.493738,3475.876262,0.702315,477.283479,-827.88,1305.163479
2,3,911,BALCAZAR JUSTINIANO MARIO,5390.78,4970.14,0.921970,100.02,66.35,33.67,0.663367,...,0.258603,114.920802,47.592543,5469.373160,3221.267361,1748.872639,0.648124,1567.549952,420.64,1146.909952


In [23]:
avance = xlsx_avance[xlsx_avance['COD_COS'] == cod_grupo]
avance

,COD_COS,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO,ENTREGA,%_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,...,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROM_REST,DIFERENCIA
0,1,515,AGUILERA OLGA RIVERO VDA DE,26449.77,26112.72,0.987257,543.39,420.85,122.54,0.77449,...,0.212767,543.693782,55.390276,30115.348382,24326.385166,1786.334834,0.931591,5788.963216,337.05,5451.913216


In [25]:
avance.columns

Index(['COD_COS', 'CODIGO CAÑERO', 'NOMBRE CAÑERO', 'COMPROMISO', 'ENTREGA',
       '%_ENTREGA', 'TOTAL_AREA', 'COSECHA', 'SIN COSECHA', '%COSECHA', 'TCH',
       'DIF.AVANCE', 'AREA_ESTIM.', 'TCH_ESTIM', 'PRODUCCION_ESTIM',
       'TN_COSECHA_ESTIM', 'DIF.ESTIM', '% DIF.ESTIM', 'TN_EN_CAMPO',
       'COMPROM_REST', 'DIFERENCIA'],
      dtype='object')

In [35]:
avance_totales = {
    'fecha_aclaracion' : fecha_aclaracion,
    'fecha_informe' : fecha_inf,
    
    'entrega_porcen' : float(avance['%_ENTREGA']) * 100,
    'ha_canha' : float(avance['TOTAL_AREA']),
    'ha_cosecha' : float(avance['COSECHA']),
    'cosecha_porcen' : float(avance['%COSECHA']) * 100,
    'diferencia' : float(avance['DIF.AVANCE']) * 100,
    
    'compromiso_rest' : float(avance['COMPROM_REST']),
    'tn_en_campo' : float(avance['TN_EN_CAMPO']),
    'diferencia' : float(avance['DIFERENCIA'])
}
avance_totales

{'fecha_aclaracion': '10 de septiembre de 2023',
 'fecha_informe': '12 de septiembra de 2023',
 'entrega_porcen': 98.72569780379943,
 'ha_canha': 543.39,
 'ha_cosecha': 420.85,
 'cosecha_porcen': 77.44897771398075,
 'diferencia': 5451.913216092134,
 'compromiso_rest': 337.0499999999993,
 'tn_en_campo': 5788.963216092133}

In [36]:
obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
obj_avance_totales

avance_totales(fecha_aclaracion='10 de septiembre de 2023', fecha_informe='12 de septiembra de 2023', entrega_porcen=98.72569780379943, ha_canha=543.39, ha_cosecha=420.85, cosecha_porcen=77.44897771398075, diferencia=5451.913216092134, compromiso_rest=337.0499999999993, tn_en_campo=5788.963216092133)

In [68]:
doc = DocxTemplate(ruta + '/templates/tbl_rpt_posible_desvio.docx')
context = {
    'insp' : list_obj_caneros,
    'grupo_totales' : obj_grupo_totales,
    'avance' : obj_avance_totales
}
doc.render(context)
doc.save(ruta + '/informes/rpt.docx')